<a href="https://colab.research.google.com/github/KadaliSana/13B-Alpaca-colab/blob/main/AlpacaLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers -y
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install flask_ngrok
!pip install ngrok
!pip install pyngrok
!ngrok authtoken '1atGT0XTeO5twH4M9hQhpUyLQCC_2ugMyUcj1DwJeGJ6zKUcn'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83

In case GPU memory is less than required to be used then change the model from 13B to 7B by replacing the 13 by 7 which uses only 6gb of vram.




In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-13b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-13b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca13B-lora")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
from flask_ngrok import run_with_ngrok
import flask

app = flask.Flask(__name__)

@app.route('/<text>', methods= ['GET'])
def alpaca_talk(text):
    text = '''Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:''' + text +'''### Response:'''
    output = text.replace("%20"," ")
    print (output)
    inputs = tokenizer(
        output,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        response = (tokenizer.decode(s))
        print (response)
        return response


input_text ='''
What are Alpacas and how are they different to Lamas?
'''

alpaca_talk(input_text)

run_with_ngrok(app)

if __name__ == '__main__':
    app.run()  

Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:
What are Alpacas and how are they different to Lamas?
### Response:
Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:
What are Alpacas and how are they different to Lamas?
### Response: 
Alpaca (Vicugna pacos) are South American camelids, related to llamas but smaller in size with shorter ears and necks. They have long silky coats which come in many colors including white, black, brown, grey or fawn. Their wool can be used for making clothing such as sweaters, hats, scarves and blankets.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dde3-35-240-207-150.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:hello### Response:
Generating...


INFO:werkzeug:127.0.0.1 - - [10/May/2023 10:09:55] "GET /hello HTTP/1.1" 200 -


 Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:hello### Response:Hello!
Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:favicon.ico### Response:
Generating...


INFO:werkzeug:127.0.0.1 - - [10/May/2023 10:10:00] "GET /favicon.ico HTTP/1.1" 200 -


 Below is an instruction that describes a task. Write a response that appropriately completes the request.
     ### Instruction:favicon.ico### Response:https://www.example.com/favicon.ico


### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca's are small, domesticated animals native to South America while Lama's are large wild animals from Central Asia. 
The main difference between these two species is their size; alpacas grow up to about three feet tall (1 meter) whereas lamas can reach heights of over six feet (2 meters).

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:
'''

alpaca_talk(input_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:

Generating...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 9>:9                                                                              │
│ in alpaca_talk:23                                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:739 in generate                       │
│                                                                                                  │
│    736 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    737 │   │   try:                                                                              │
│    738 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  739 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    740 │   │   │   else:                                                                         │
│    741 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    742 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1406 in generate        │
│                                                                                                  │
│   1403 │   │   │   │   )                                                                         │
│   1404 │   │   │                                                                                 │
│   1405 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1406 │   │   │   return self.greedy_search(                                                    │
│   1407 │   │   │   │   input_ids,                                                                │
│   1408 │   │   │   │   logits_processor=logits_processor,                                        │
│   1409 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2201 in greedy_search   │
│                                                                                                  │
│   2198 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2199 │   │   │                                                                                 │
│   2200 │   │   │   # forward pass to get next token                                              │
│ ❱ 2201 │   │   │   outputs = self(                         

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email to OpenAI saying why GPT4 should be open-source.

### Response:
'''

alpaca_talk(input_text)

### Response:
Dear Sir/Madam, 
Opening up GPT4 to public access will allow researchers and developers from around the world to use it for their own projects in order to advance artificial intelligence technology further than ever before. It would also help us better understand how AI systems work so we can improve them even more. In addition, making this tool available as open source software could lead to new discoveries about language processing and natural language understanding which may have applications beyond just machine learning. I believe these are all reasons why GPT4 should be made open source


In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
'''

alpaca_talk(input_text)

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like humans have all of the intelligence, alpaca's actually possess much more advanced cognitive abilities than we do! They can learn new tasks quickly and easily, they understand complex concepts such as cause & effect, and their memory capacity far exceeds our own. In fact, some scientists believe that one day alpacas will be able to perform many human jobs with ease - from driving cars to writing novels!

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
'''

alpaca_talk(input_text)

### Instruction:
Tell me a joke about an Alpaca

### Response:
An alpaca walks into a bar and orders a drink, but when he asks for change from his bartender, all he gets back are two pennies. The alpaca says "That's not fair! I should get more than just 2 cents!"